<a href="https://colab.research.google.com/github/DataSavvyYT/experiments/blob/main/playwright/1_visit_a_Site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Swarm from OpenAI's GitHub and other dependencies
!pip install -q git+https://github.com/openai/swarm.git playwright nest_asyncio

# Install Playwright browsers and system dependencies
!playwright install chromium
!playwright install-deps

In [ ]:
import os
import getpass
import asyncio
import nest_asyncio
from swarm import Swarm, Agent
from playwright.async_api import async_playwright
from openai import OpenAI

# 1. Apply nest_asyncio to allow nested event loops in Colab
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
# 3. Define the Tool
# Swarm tools are just Python functions. We wrap the async Playwright code here.
def get_website_title(url):
    """
    Visits a website using a headless browser and returns its title.
    """
    print(f"DEBUG: Agent is visiting {url}...")

    async def run_browser():
        async with async_playwright() as p:
            # Launch headless browser
            browser = await p.chromium.launch(headless=True, args=["--disable-gpu", "--no-sandbox"])
            page = await browser.new_page()

            try:
                await page.goto(url)
                title = await page.title()
                return title
            except Exception as e:
                return f"Error: {e}"
            finally:
                await browser.close()

    # Execute the async function synchronously for Swarm
    return asyncio.get_event_loop().run_until_complete(run_browser())

In [ ]:
# 4. Initialize Swarm Client
# We need to instantiate the OpenAI client first.
openai_client = OpenAI(api_key=OPENAI_API_KEY)
client = Swarm(openai_client)

In [ ]:
# 5. Create the Agent
# We give it our tool and instructions.
browser_agent = Agent(
    name="BrowserAgent",
    model="gpt-4o-mini",
    instructions="You are a helpful web assistant. Use the available tools to browse the web when asked.",
    functions=[get_website_title]
)

In [ ]:
# 6. Run the Agent
print("--- Starting Agent ---")
response = client.run(
    agent=browser_agent,
    messages=[{"role": "user", "content": "Go to https://www.google.com and tell me what the title is."}]
)

In [ ]:
# 7. Print Result
print("\n--- Final Response ---")
print(response.messages[-1]["content"])